In [1]:
import matplotlib.pyplot as plt


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / "_scripts"))

In [52]:
import networkx as nx
from eppy.bunch_subclass import EpBunch
from geomeppy import IDF
import seaborn as sns
import polars as pl
import numpy as np

from case_edits.ezcase import *

from helpers.variables import *
from helpers.variable_interfaces import *
from setup.setup import get_case_names, CaseData, retrieve_cases, create_cases, retrieve_paths
from setup.plots import *
from setup.data_wrangle import *
from network.data_on_network import *

import matplotlib.cm as cm
import matplotlib as mpl


from matplotlib.colors import Colormap, Normalize, LinearSegmentedColormap
from setup.setup import create_directories

from helpers.ep_helpers import WallNormal, get_partner_of_surface
from subsurfaces.creator import SubsurfaceCreator
from airflow_network.airboundary import *
from subsurfaces.logic import find_surface_connecting_two_zones

In [5]:
# pairs = get_subsurface_pairs_from_case(path_to_inputs)
# pairs

In [56]:
_, bol, _ = create_directories()
input_dir = "case_bol_5"
output_dir = "tests/test23_airwall"
path_to_inputs, path_to_outputs = retrieve_paths(input_dir, output_dir)
case = initialize_case(path_to_outputs)
case.idf = add_rooms(case.idf, path_to_inputs)
case

EPCaseEditor(test23_airwall)

In [48]:
o = case.idf.newidfobject("CONSTRUCTION:AIRBOUNDARY")
o.Name = f"AirBoundary"

ab_pairs = get_airboundary_pairs_from_case(path_to_inputs)
for pair in ab_pairs:
    surf = find_surface_connecting_two_zones(case.idf, pair)
    assert surf
    surf.Construction_Name = o.Name 
    partner_surf = get_partner_of_surface(case.idf, surf)
    assert partner_surf
    partner_surf.Construction_Name = o.Name

In [57]:
case.idf = update_air_boundary_constructions(case.idf, path_to_inputs)

In [58]:
case.idf.getsurfaces()

[
 BUILDINGSURFACE:DETAILED,
     Block 00 Storey 0 Wall 0002,    !- Name
     wall,                     !- Surface Type
     Project Wall,             !- Construction Name
     Block 00 Storey 0,        !- Zone Name
     ,                         !- Space Name
     outdoors,                 !- Outside Boundary Condition
     ,                         !- Outside Boundary Condition Object
     SunExposed,               !- Sun Exposure
     WindExposed,              !- Wind Exposure
     autocalculate,            !- View Factor to Ground
     autocalculate,            !- Number of Vertices
     2.5,                      !- Vertex 1 Xcoordinate
     6.44,                     !- Vertex 1 Ycoordinate
     3.05,                     !- Vertex 1 Zcoordinate
     2.5,                      !- Vertex 2 Xcoordinate
     6.44,                     !- Vertex 2 Ycoordinate
     0,                        !- Vertex 2 Zcoordinate
     0,                        !- Vertex 3 Xcoordinate
     6.44,          

In [59]:
case.idf = add_subsurfaces(case.idf, path_to_inputs)
case.idf = add_airflownetwork(case.idf)

case.idf = add_all_output_requests(case.idf)
case.compare_and_save()

out.idf does not exist


In [60]:
all_subsurf = [i.Name for i in case.idf.getsubsurfaces()]
len(set(all_subsurf)) == len(all_subsurf)

True

In [61]:
case.run_idf()

idf has changed - running case

/Applications/EnergyPlus-22-2-0/energyplus --weather /Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/weather_data/PALO_ALTO/CA_PALO-ALTO-AP_724937_23.EPW --output-directory /Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/cases/tests/test23_airwall/results --idd /Applications/EnergyPlus-22-2-0/Energy+.idd /Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/cases/base/01example/Minimal_AP_fda4a3.idf

EnergyPlus Starting
EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.10.30 07:56
Adjusting Air System Sizing
Adjusting Standard 62.1 Ventilation Sizing
Initializing Simulation
Reporting Surfaces
Beginning Primary Simulation
Initializing New Environment Parameters
Warming up {1}
Warming up {2}
Warming up {3}
Warming up {4}
Warming up {5}
Warming up {6}
Starting Simulation at 07/01/2017 for SUMMERDAY
Writing tabular output file results using HTML format.
Writing final SQL reports
EnergyPlus Run Time=00hr 00min  0.37sec


EnergyPlus Completed Successfully.
